In [1]:
# Import relevant modules
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load a pretrained model and tokenizer (e.g., BERT for sequence classification)
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

2025-08-25 14:58:57.122044: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-25 14:58:57.302962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756126737.367818   13109 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756126737.386802   13109 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756126737.513026   13109 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the IMDb dataset
dataset = load_dataset('imdb')

# Convert dataset to Pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Perform train-test split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Convert back to Hugging Face dataset
from datasets import Dataset
train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

# Apply preprocessing
train_data = train_data.map(preprocess_function, batched=True)
test_data = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [4]:
import os
import torch
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification

# Disable parallelism warning and MLflow logging
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["MLFLOW_TRACKING_URI"] = "disable"
os.environ["HF_MLFLOW_LOGGING"] = "false"

# Ensure CPU usage if no GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a smaller, faster model like DistilBERT
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

"""
# Use a subset of the dataset to speed up training
train_data = train_data.select(range(1000))  # Select 1000 samples for training
test_data = test_data.select(range(200))     # Select 200 samples for evaluation
"""

# Set up training arguments for faster training
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="steps",
    eval_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=8,   
    num_train_epochs=1,              
    weight_decay=0,                  
    logging_steps=500,               
    save_steps=1000,                 
    save_total_limit=1,              
    gradient_accumulation_steps=1,   
    fp16=False,                      
    report_to="none",                
)

# Define the Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

# Fine-tune the model
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
500,0.356700,0.255557
1000,0.308600,0.237554
1500,0.285100,0.226803
2000,0.275000,0.226311
2500,0.247000,0.243549


TrainOutput(global_step=2500, training_loss=0.29449859313964843, metrics={'train_runtime': 932.9657, 'train_samples_per_second': 21.437, 'train_steps_per_second': 2.68, 'total_flos': 2649347973120000.0, 'train_loss': 0.29449859313964843, 'epoch': 1.0})

In [5]:
# Evaluate the model
results = trainer.evaluate()

# Print evaluation results
print(f"Accuracy: {results['eval_accuracy']}")

KeyError: 'eval_accuracy'

In [6]:
results

{'eval_loss': 0.24354904890060425,
 'eval_runtime': 50.4874,
 'eval_samples_per_second': 99.035,
 'eval_steps_per_second': 12.379,
 'epoch': 1.0}